In [2]:
import pandas as pd
import duckdb

In [3]:
df = pd.read_csv('./place_fips.txt', delimiter='|')
df['STATEFP'] = df['STATEFP'].astype(str)
df['PLACEFP'] = df['PLACEFP'].astype(str)

In [4]:
query = "SELECT * FROM df limit 10"
duckdb.query(query).df()

,STATE,STATEFP,PLACEFP,PLACENS,PLACENAME,TYPE,CLASSFP,FUNCSTAT,COUNTIES
0,AL,1,100,2582661,Abanda CDP,CENSUS DESIGNATED PLACE,U1,S,Chambers County
1,AL,1,124,2403054,Abbeville city,INCORPORATED PLACE,C1,A,Henry County
2,AL,1,460,2403063,Adamsville city,INCORPORATED PLACE,C1,A,Jefferson County
3,AL,1,484,2405123,Addison town,INCORPORATED PLACE,C1,A,Winston County
4,AL,1,676,2405125,Akron town,INCORPORATED PLACE,C1,A,Hale County
5,AL,1,820,2403069,Alabaster city,INCORPORATED PLACE,C1,A,Shelby County
6,AL,1,988,2403074,Albertville city,INCORPORATED PLACE,C1,A,Marshall County
7,AL,1,1132,2403077,Alexander City city,INCORPORATED PLACE,C1,A,Tallapoosa County
8,AL,1,1180,2402638,Alexandria CDP,CENSUS DESIGNATED PLACE,U1,S,Calhoun County
9,AL,1,1228,2403080,Aliceville city,INCORPORATED PLACE,C1,A,Pickens County


In [5]:
duckdb.sql("SELECT * FROM df limit 10")

┌─────────┬─────────┬─────────┬─────────┬───┬──────────────────────┬─────────┬──────────┬───────────────────┐
│  STATE  │ STATEFP │ PLACEFP │ PLACENS │ … │         TYPE         │ CLASSFP │ FUNCSTAT │     COUNTIES      │
│ varchar │ varchar │ varchar │  int64  │   │       varchar        │ varchar │ varchar  │      varchar      │
├─────────┼─────────┼─────────┼─────────┼───┼──────────────────────┼─────────┼──────────┼───────────────────┤
│ AL      │ 1       │ 100     │ 2582661 │ … │ CENSUS DESIGNATED …  │ U1      │ S        │ Chambers County   │
│ AL      │ 1       │ 124     │ 2403054 │ … │ INCORPORATED PLACE   │ C1      │ A        │ Henry County      │
│ AL      │ 1       │ 460     │ 2403063 │ … │ INCORPORATED PLACE   │ C1      │ A        │ Jefferson County  │
│ AL      │ 1       │ 484     │ 2405123 │ … │ INCORPORATED PLACE   │ C1      │ A        │ Winston County    │
│ AL      │ 1       │ 676     │ 2405125 │ … │ INCORPORATED PLACE   │ C1      │ A        │ Hale County       │
│ AL      

PLACE = STATE+PLACE = 2+5 = 7 code digits

In [6]:
query = """
SELECT 
* 
FROM df 
where PLACENAME LIKE 'Dallas%'
and STATE = 'TX'
limit 15
"""
duckdb.query(query).df()

,STATE,STATEFP,PLACEFP,PLACENS,PLACENAME,TYPE,CLASSFP,FUNCSTAT,COUNTIES
0,TX,48,19000,2410288,Dallas city,INCORPORATED PLACE,C1,A,Collin County~~~Dallas County~~~Denton County~...


In [7]:
query = """
SELECT 
* 
FROM df 
where (LENGTH(PLACEFP) = 5
or LENGTH(PLACEFP) = 4)
and STATE = 'TX'
and PLACENAME LIKE 'Arlington%'
limit 15
"""
duckdb.query(query).df()

,STATE,STATEFP,PLACEFP,PLACENS,PLACENAME,TYPE,CLASSFP,FUNCSTAT,COUNTIES
0,TX,48,4000,2409731,Arlington city,INCORPORATED PLACE,C1,A,Tarrant County


Now filter and export csv file with 7-digit place fips

In [8]:
query = """
select 
STATE
, STATEFP
, PLACEFP
, CONCAT(STATEFP, LPAD(PLACEFP, 5, '0')) as FIPS 
, PLACENAME
, COUNTIES 
from df 
where (
    LENGTH(PLACEFP) = 5
    or LENGTH(PLACEFP) = 4
    )
order by PLACEFP asc
limit 15
"""
duckdb.query(query).df()

,STATE,STATEFP,PLACEFP,FIPS,PLACENAME,COUNTIES
0,ID,16,1000,1601000,Albion city,Cassia County
1,KY,21,1000,2101000,Allensville CDP,Todd County
2,IA,19,1000,1901000,Alburnett city,Linn County
3,FL,12,1000,1201000,Altha town,Calhoun County
4,WI,55,1000,5501000,Algoma city,Kewaunee County
5,VA,51,1000,5101000,Alexandria city,Alexandria city
6,TX,48,1000,4801000,Abilene city,Jones County~~~Taylor County
7,OR,41,1000,4101000,Albany city,Benton County~~~Linn County
8,OH,39,1000,3901000,Akron city,Summit County
9,NC,37,1000,3701000,Alliance town,Pamlico County


In [9]:
query = """
select 
STATE as state
, LPAD(STATEFP, 2, '0') as statefp
, PLACEFP as placefp
, CONCAT(LPAD(STATEFP, 2, '0'), LPAD(PLACEFP, 5, '0')) as fips
, PLACENAME as placename
, COUNTIES as counties
from df 
where (LENGTH(PLACEFP) = 5
or LENGTH(PLACEFP) = 4)
"""
place_fips_df = duckdb.query(query).df()
place_fips_df.head(10)

,state,statefp,placefp,fips,placename,counties
0,AL,01,10240,0110240,Brundidge city,Pike County
1,AL,01,10456,0110456,Bucks CDP,Mobile County
2,AL,01,11032,0111032,Butler town,Choctaw County
3,AL,01,11416,0111416,Calera city,Chilton County~~~Shelby County
4,AL,01,11488,0111488,Calvert CDP,Mobile County~~~Washington County
5,AL,01,11512,0111512,Camden city,Wilcox County
6,AL,01,11680,0111680,Camp Hill town,Tallapoosa County
7,AL,01,12016,0112016,Carbon Hill city,Walker County
8,AL,01,12040,0112040,Cardiff town,Jefferson County
9,AL,01,12070,0112070,Carlisle-Rockledge CDP,Etowah County


In [10]:
query = """
select
*
from place_fips_df
where fips = '4837000'
"""
duckdb.query(query).df()

,state,statefp,placefp,fips,placename,counties
0,TX,48,37000,4837000,Irving city,Dallas County


In [11]:
place_fips_df.to_csv('../data/place_fips.csv', index=False)